In [20]:
import pandas as pd
from tqdm import tqdm
import re

In [43]:
def Get_year_results(year_string, out_string):
    results = pd.read_html('https://sport.mkez.tw/games/'+ year_string +'/front/schedule_ball_score.php')[0]
    
    for i in tqdm(range(1,150)):
        results_temp = pd.read_html('https://sport.mkez.tw/games/'+ year_string +'/front/schedule_ball_score.php?pageNum_RS=' + str(i))[0]
        #print(html_tables_temp)
        results = pd.concat([results, results_temp])
        
    results.columns = ['matchID', 'Time', 'Group', 'inGroup_matchID', 'Player1', 'Score', 'Player2', 'Ranking']
    results = results.reset_index(drop = True)
    invalid_scores = ['傷', '空', '請', '棄', '-2', '-5']

    for i in invalid_scores:
        f = results['Score'].str.contains(i)
        results = results[~f]

    results['Gr'] = results['Group'].str.replace("子組第.次賽 圖", "", regex = True)
    results['Round'] = results['Group'].str.replace("(.子組| 圖)", "", regex = True)

    player_stats = {}
    for i in results.index:
        #player 1
        Player_entry = re.sub("(\\(.+\\)|[0-9]|勝)", "", results.Player1[i])
        Player_name = Player_entry.split(' - ')[1].strip() + ' - ' + Player_entry.split(' - ')[0]
    
        selection = '-'
        score = results.Score[i].split(' : ')
        games_W = int(score[0])
        games_L = int(score[1])
        
        if games_W == 3:
            match_W = 1
            match_L = 0
            if '入選' in str(results.Ranking[i]):
                selection = results.Round[i] + '入選'
        else:
            match_W = 0
            match_L = 1
    
    
        if Player_name not in player_stats.keys():
            player_stats[Player_name] = {'name': Player_name, 'Gr': results.Gr[i],
                                         'W': match_W, 'L': match_L,
                                         'g_W': games_W, 'g_L': games_L, 
                                         'selection': selection
                                        }
        else:
            player_stats[Player_name]['W'] += match_W
            player_stats[Player_name]['L'] += match_L
            player_stats[Player_name]['g_W'] += games_W
            player_stats[Player_name]['g_L'] += games_L
            if '入選' in selection:
                player_stats[Player_name]['selection'] = selection
        
        #player 2
        Player_entry = re.sub("(\\(.+\\)|[0-9]|勝)", "", results.Player2[i])
        Player_name = Player_entry.split(' - ')[1].strip() + ' - ' + Player_entry.split(' - ')[0]
        selection = '-'
        
        score = results.Score[i].split(' : ')
        games_W = int(score[1])
        games_L = int(score[0])
    
        if games_W == 3:
            match_W = 1
            match_L = 0
            if '入選' in str(results.Ranking[i]):
                selection = results.Round[i] + '入選'
        else:
            match_W = 0
            match_L = 1
        
        if Player_name not in player_stats.keys():
            player_stats[Player_name] = {'name': Player_name, 'Gr': results.Gr[i],
                                         'W': match_W, 'L': match_L,
                                         'g_W': games_W, 'g_L': games_L,
                                         'selection': selection
                                        }
        else:
            player_stats[Player_name]['W'] += match_W
            player_stats[Player_name]['L'] += match_L
            player_stats[Player_name]['g_W'] += games_W
            player_stats[Player_name]['g_L'] += games_L
            
            if '入選' in selection:
                player_stats[Player_name]['selection'] = selection

    player_stats = pd.DataFrame.from_dict(player_stats, orient='index').reset_index(drop = True)
    player_stats['Tot'] = player_stats.W + player_stats.L
    player_stats['W_pct'] = round(player_stats.W / player_stats.Tot, 3)
    player_stats['Tot_Gms'] = player_stats.g_W + player_stats.g_L
    player_stats['g_W_pct'] = round(player_stats.g_W / player_stats.Tot_Gms, 3)
    player_stats = player_stats.sort_values(by=['W_pct', 'g_W_pct'], ascending = False)
    player_stats = player_stats[['name', 'Gr', 
                                 'W', 'L', 'Tot',  'W_pct', 'selection',
                                 'g_W', 'g_L',  'Tot_Gms', 'g_W_pct',]]
    
    player_stats.to_csv(out_string + '_results.tsv', sep="\t", index = False, float_format="%.3f") 
    
    player_stats_f = player_stats[player_stats['Gr'].str.contains('女')]
    player_stats_f.to_csv(out_string + '_results_f.tsv', sep="\t", index = False, float_format="%.3f") 
    
    player_stats_m = player_stats[player_stats['Gr'].str.contains('男')]
    player_stats_m.to_csv(out_string + '_results_m.tsv', sep="\t", index = False, float_format="%.3f") 

    return None


In [ ]:
Get_year_results('tt_11112', '112')

 79%|█████████████████████████████████████████████████████████████████████████                    | 117/149 [00:30<00:10,  2.98it/s]

In [ ]:
Get_year_results('tt_1131112', '114')